__算术编码__

将整个符号序列映射为实轴上[0,1)内的一个小区间，区间长度等于该序列的概率，再在小区间内选择一个代表性的二进制小数作为编码输出。

__多元符号编码原理__

设输入符号串s取自符号集S={(a_1,p_1),(a_2,p_2)...(a_m,p_m)}，s后跟符号ai扩展成符号串sai，空串记作Φ，迭代关系可表示为:

    1. 码字刷新 左边界C(sa_i) = C(s) + P(a_i)A(s)

    2. 区间刷新 区间宽度A(sa_i) = p(a_i)A(s) 

    其中P(a_i) = p(a_1)+p(a_2)+...+p(a_(i-1))，是符号的累积概率。初始条件C(Φ)=0，A(Φ)=1，P(Φ)=0，p(Φ)=1

In [1]:
sym = ['a', 'b', 'c', 'd', 'e', '!'];
p = [0.2, 0.1, 0.1, 0.3, 0.2, 0.1];
seq = "dead!";
C = 0;
A = 1;
P = 0;
for i in seq
    index = findfirst(sym.==i);
    if index > 1
        P = sum([p[j] for j in 1:index-1]);
    else
        P = 0;
    end
    C = C + P*A;
    A = p[index]*A;
end
println((C, C+A))

(0.61804, 0.6184000000000001)


__二进制编码__

算数编码每次递推都要做乘法，如果编码对象本身是二元序列，且符号概率较小者为p(L)=2^(-Q)的形式(Q为正整数)，则乘以2^(-Q)可用右移Q为代替，符号概率较大者p(H)=1-2^(-Q)可用移位和减法代替，从而避免乘法运算。因此算数编码很适合二元序列，p(L)常用2^(-Q)近似。

编码迭代公式变为：

    1.码字刷新 C = C + P(a_i)*A

    2.区间刷新 A = p(a_i)*A

令S={H,L}，并设p(L)=2^(-Q)，p(H)=1-2^(-Q);则P(H)=0,P(L)=1-2^(-Q)，从而有限精度、不做乘法且假设接收符号概率Q(s)已经估计出的二进制算数编码的具体步骤如下：

    1.初始化 C=0.0...0,A=0.1...1(小数点后保留q位)

    2.对子区间宽度A(s)做迭代运算: A(sL)=A(s)*2(-Q(s)) A(sH) = <A(s) - A(sL)> (<X>表示X的小数点后取q位)

    3.对码字C(s)做迭代运算: C(sH) = C(s) C(sL) = C(s) + A(sH)

    4.如果得到的新码字A(sx)<0.10...0，则A、C重复左移直到A>=0.10...0为止，保持A(s)的小数点后第1位始终为1

    5.如果紧靠C的小数点前由连续v个1，则紧靠小数点前插入1个0作为填充位

    6.按上述步骤对字符串中所有字符进行迭代运算，直到最后一个字符输出C(s)代码

In [34]:
seq = "01000101";
Q = [2,1,2,2,3,1,1,2];
q = 4;
v = 3;
C, A, Asl, Ash = 0, 1-1/2^q, 1-1/2^q, 1-1/2^q;
for (i,value) in enumerate(seq)
    Asl = floor(A*2^(q-Q[i]))/2^q;
    Ash = floor((A - Asl)*2^q)/2^q;
    if value == '1'
        C = C + Ash;
        A = Asl;
    else
        A = Ash;
    end
    while A < 0.5 && A != 0
        A = A * 2;
        C = C * 2;
    end
    if floor(C) % 2^v == (2^v-1)
        C = C + floor(C);
    end
end
println(A," ",C)
println(string(Integer(floor(C)),base=2), ".", string(Integer((C-floor(C))*2^q),base=2,pad=4))

0.75 61.25
111101.0100
